## Import's

In [1]:
# Базовые
import numpy as np
import pandas as pd
from scipy import sparse

# Предобработка и пайплайны
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Метрики
from sklearn.metrics import (
    roc_auc_score,
)

from sklearn.tree import DecisionTreeClassifier as SkDecisionTreeClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

## 1. Download data from Don’tGetKicked competition.Design train/validation/test split.

Use the "PurchDate" field to split, test must be later in time than validation, same goes for validation and train: train.PurchDate < valid.PurchDate < test.PurchDate.

Use the first 33% of the data for the training, the last 33% of the data for the test, and the middle 33% for the validation set. Don't use the test dataset until the end!

Use LabelEncoder or OneHotEncoder from sklearn to preprocess categorical variables. Be careful with data leakage (fit Encoder to training and apply to validation & test). 

### - Download data from Don’tGetKicked competition.

In [2]:
df = pd.read_csv("./data/training.csv")

In [3]:
df.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


### - Design the train/validation/test split. Use the "PurchDate" field for the split, test must be later than validation, same for validation and train: train.PurchDate < valid.PurchDate < test.PurchDate. Use the first 1/3 of dates for the train, the last 1/3 of dates for the test, and the middle 1/3 for the validation set. Don’t use the test dataset until the end!

In [4]:
def split_train_val_test_date(df: pd.DataFrame, date_col: str = "PurchDate", y_col: str = "IsBadBuy") -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
  d = df.copy()
  d[date_col] = pd.to_datetime(d[date_col])

  uniq_dates = np.array(sorted(d[date_col].unique()))
  n = len(uniq_dates)

  a = n // 3
  b = 2 * n // 3

  d_train = uniq_dates[:a]
  d_valid = uniq_dates[a:b]
  d_test = uniq_dates[b:]

  train_df = d[d[date_col].isin(d_train)].sort_values(date_col)
  valid_df = d[d[date_col].isin(d_valid)].sort_values(date_col)
  test_df = d[d[date_col].isin(d_test)].sort_values(date_col)

  return train_df, valid_df, test_df

In [5]:
train_df, valid_df, test_df = split_train_val_test_date(df)

In [6]:
train_df["PurchDate"].max() < valid_df["PurchDate"].min() < test_df["PurchDate"].min()

True

In [7]:
display(train_df, valid_df, test_df)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
32387,32409,0,2009-01-05,MANHEIM,2004,5,FORD,TAURUS 3.0L V6 EFI,SES,4D SEDAN SES DURATEC,...,4139.0,5351.0,NaN,NaN,22916,80022,CO,4900.0,0,825
32386,32408,0,2009-01-05,MANHEIM,2006,3,CHEVROLET,TRAILBLAZER EXT 4WD,LS,4D SUV 4.2L,...,10438.0,12158.0,NaN,NaN,22916,80022,CO,8180.0,0,1703
32385,32407,0,2009-01-05,MANHEIM,2004,5,DODGE,STRATUS 4C 2.4L I4 M,SE,4D SEDAN SE,...,4169.0,5114.0,NaN,NaN,3453,80022,CO,4250.0,0,1155
32384,32406,0,2009-01-05,MANHEIM,2005,4,FORD,FREESTAR FWD V6 3.9L,SES,4D PASSENGER 3.9L SES,...,5801.0,6949.0,NaN,NaN,22916,80022,CO,6160.0,0,941
32383,32405,0,2009-01-05,MANHEIM,2005,4,PONTIAC,GRAND PRIX 3.8L V6 S,Bas,4D SEDAN,...,6670.0,8017.0,NaN,NaN,22916,80022,CO,6010.0,0,1974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56842,56870,1,2009-09-01,MANHEIM,2004,5,FORD,EXPLORER 2WD V6 4.0L,XLT,4D SUV 4.0L XLT,...,8813.0,10205.0,NaN,NaN,23359,92504,CA,7520.0,0,1155
56843,56871,0,2009-09-01,MANHEIM,2008,1,DODGE,AVENGER 4C 2.4L I4 S,SE,4D SEDAN,...,9753.0,10571.0,NaN,NaN,23359,92504,CA,7855.0,0,1020
56844,56872,0,2009-09-01,MANHEIM,2007,2,CHRYSLER,PT CRUISER 2.4L I4 S,Bas,4D SEDAN,...,7169.0,8141.0,NaN,NaN,23359,92504,CA,6140.0,0,1215
56834,56862,0,2009-09-01,MANHEIM,2008,1,DODGE,AVENGER 4C 2.4L I4 S,SE,4D SEDAN,...,11777.0,12505.0,NaN,NaN,23359,92504,CA,7755.0,0,1020


,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
69724,69756,0,2009-09-02,ADESA,2006,3,CHRYSLER,300 2.7L V6 MPI,Bas,4D SEDAN,...,11680.0,13502.0,NaN,NaN,21053,85226,AZ,9795.0,0,1215
50519,50547,0,2009-09-02,MANHEIM,2006,3,DODGE,CARAVAN FWD 4C 2.4L,SE,MINIVAN 2.4L,...,7299.0,8923.0,NaN,NaN,20833,78219,TX,6150.0,0,1411
50518,50546,0,2009-09-02,MANHEIM,2006,3,CHRYSLER,PT CRUISER 2.4L I4 S,Lim,4D SEDAN,...,8126.0,9545.0,NaN,NaN,18822,78219,TX,6735.0,0,1086
50517,50545,0,2009-09-02,MANHEIM,2002,7,MITSUBISHI,MONTERO 4WD V6 3.5L,Lim,4D SPORT UTILITY LIMITED,...,8068.0,9086.0,NaN,NaN,18822,78219,TX,7135.0,0,1209
484,485,0,2009-09-02,ADESA,2004,5,PONTIAC,GRAND AM V6 3.4L V6,SE,4D SEDAN SE1,...,3939.0,5173.0,NaN,NaN,19619,20166,VA,5685.0,0,1666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48619,48643,1,2010-04-30,MANHEIM,2006,4,CHRYSLER,PT CRUISER,Tou,4D SEDAN,...,9005.0,10247.0,NaN,NaN,16044,17545,PA,5785.0,0,1389
48620,48644,0,2010-04-30,MANHEIM,2006,4,DODGE,STRATUS V6,SXT,4D SEDAN,...,7666.0,8624.0,NaN,NaN,16044,17545,PA,5940.0,0,1389
48621,48645,0,2010-04-30,MANHEIM,2004,6,CHEVROLET,MALIBU MAXX V6,LS,4D SEDAN LS,...,7893.0,9339.0,NaN,NaN,16044,17545,PA,7460.0,0,1373
50880,50908,0,2010-04-30,MANHEIM,2002,8,DODGE,NEON,Bas,4D SEDAN,...,4310.0,5078.0,NaN,NaN,18822,78219,TX,4135.0,0,986


,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
234,235,0,2010-05-03,ADESA,2005,5,SATURN,ION,1,4D SEDAN LEVEL 1,...,6658.0,7586.0,NaN,NaN,5546,33619,FL,4900.0,0,853
20195,20208,0,2010-05-03,OTHER,2004,6,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6429.0,7692.0,NaN,NaN,21053,95673,CA,4990.0,0,1020
20194,20207,1,2010-05-03,OTHER,2005,5,NISSAN,ALTIMA,Bas,4D SEDAN,...,10167.0,11829.0,NaN,NaN,21053,95673,CA,7605.0,0,723
20193,20206,0,2010-05-03,OTHER,2007,3,DODGE,MAGNUM V6,SE,WAGON 2.7L,...,13464.0,15661.0,NaN,NaN,21053,95673,CA,9130.0,0,1503
20192,20205,0,2010-05-03,OTHER,2007,3,DODGE,1500 RAM PICKUP 2WD,SLT,QUAD CAB 4.7L BIG HORN,...,13781.0,17116.0,NaN,NaN,21053,95673,CA,9705.0,0,983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68071,68103,0,2010-12-30,ADESA,2006,4,CHEVROLET,IMPALA,LT,4D SEDAN LT 3.5L,...,9469.0,11362.0,NO,GREEN,52598,28273,NC,7570.0,0,1974
70414,70446,0,2010-12-30,ADESA,2005,5,CHEVROLET,TRAILBLAZER 2WD 6C,LS,4D SUV 4.2L,...,10507.0,11717.0,NO,GREEN,20740,32219,FL,8200.0,0,1155
70413,70445,0,2010-12-30,ADESA,2005,5,HYUNDAI,ELANTRA,GLS,4D SEDAN,...,6893.0,8131.0,NaN,NaN,20740,32219,FL,5050.0,0,462
68063,68095,0,2010-12-30,ADESA,2006,4,SUZUKI,AERIO,SX,4D WAGON SX,...,7601.0,8041.0,NaN,NaN,52598,28273,NC,4500.0,0,983


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

### - Use LabelEncoder or OneHotEncoder from sklearn to preprocess categorical variables. Be careful with data leakage (fit Encoder to training and apply to validation & test). Consider another coding approach if you encounter new categorical values in validation & test (not seen in training): https://contrib.scikit-learn.org/category_encoders/count.html

In [9]:
TARGET = "IsBadBuy"

In [10]:
drop_cols = [TARGET, "PurchDate"]
num_cols = train_df.select_dtypes(include=[np.number]).drop(columns=[TARGET], errors="ignore").columns.tolist()
num_cols = [c for c in num_cols if c not in drop_cols]
cat_cols = [c for c in train_df.columns if c not in num_cols + drop_cols]

In [11]:
print(f"Числовых фич: {len(num_cols)} | Категориальных фич: {len(cat_cols)}")

Числовых фич: 18 | Категориальных фич: 14


In [12]:
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler())
])

In [13]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

In [14]:
categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe",     ohe)
])

In [15]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop"
)

In [16]:
X_tr = preprocess.fit_transform(train_df)
X_va = preprocess.transform(valid_df)
X_te = preprocess.transform(test_df)

y_tr = train_df[TARGET].to_numpy(dtype=int)
y_va = valid_df[TARGET].to_numpy(dtype=int)
y_te = test_df[TARGET].to_numpy(dtype=int)

In [17]:
print("Формы матриц:",
      "\n  X_tr:", X_tr.shape,
      "\n  X_va:", X_va.shape,
      "\n  X_te:", X_te.shape)

Формы матриц: 
  X_tr: (23059, 1691) 
  X_va: (24104, 1691) 
  X_te: (25820, 1691)


In [18]:
num_names = num_cols
ohe_feature_names = preprocess.named_transformers_["cat"].named_steps["ohe"].get_feature_names_out(cat_cols).tolist()
feature_names = num_names + ohe_feature_names
print("Всего признаков после OHE:", len(feature_names))

Всего признаков после OHE: 1691


## 2. Create a Python class for Decision Tree Classifier and Decision Tree Regressor (MSE loss).

It should support fit, predict_proba, and predict methods. Also, the maximum depth (max_depth) must be a parameter of your class. Use the Gini impurity criterion as a criterion for choosing the split.

Here is the blueprint:

- model = DecisionTreeClassifier(max_depth=7)
- model.fit(Xtrain, ytrain)
- model.predict_proba(Xvalid)


Create a separate class for Node. It should be able to hold data (sample features and targets), compute Gini impurity, and have pointers to children (left and right nodes). For the Regressor, use standard deviation instead of Gini impurity.
Implement a function that finds the best possible split in the current node.
Combine the previous steps into your working Decision Tree Classifier.
Implement an Extra Randomized Tree by designing another function to find the best split.


In [19]:
def gini_from_proba(y_true, y_score):
    return 2.0 * roc_auc_score(y_true, y_score) - 1.0

In [20]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=7):
        self.max_depth = max_depth

    def fit(self, Xtrain, ytrain):
        pass
    
    def predict_proba(self, Xvalid):
        pass

In [21]:
class Node:
    def __init__(self, X=None, y=None, depth=0):
        self.X = X
        self.y = y
        self.depth = depth

        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None

        self.is_leaf = False
        self.value = None

    def gini(self):
        if self.y is None or len(self.y) == 0:
            return 0.0
        _, counts = np.unique(self.y, return_counts=True)
        p = counts / counts.sum()
        return 1.0 - np.sum(p ** 2)

    def variance(self):
        if self.y is None or len(self.y) == 0:
            return 0.0
        return np.var(self.y)

        

In [22]:
class BaseDecisionTree:
    def __init__(
        self,
        max_depth=7,
        min_samples_split=2,
        max_features=None,
        random_state=None,
        splitter="best",
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.splitter = splitter

        self.root_ = None
        self.n_features_ = None
        self.rng_ = np.random.RandomState(random_state)

    def _impurity(self, y):
        raise NotImplementedError

    def _leaf_value(self, y):
        raise NotImplementedError

    def _prepare_target(self, y):
        return y

    def _get_n_features_to_use(self):
        if self.max_features is None:
            return self.n_features_
        if isinstance(self.max_features, str):
            if self.max_features == "sqrt":
                return max(1, int(np.sqrt(self.n_features_)))
            if self.max_features == "log2":
                return max(1, int(np.log2(self.n_features_)))
            # можно добавить другие варианты
        if isinstance(self.max_features, int):
            return min(self.max_features, self.n_features_)
        # fallback
        return self.n_features_

    def _choose_features(self):
        n_feats = self._get_n_features_to_use()
        if n_feats == self.n_features_:
            return np.arange(self.n_features_)
        return self.rng_.choice(self.n_features_, size=n_feats, replace=False)

    def _best_split_exhaustive(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        best_feature = None
        best_threshold = None

        current_impurity = self._impurity(y)
        best_impurity = current_impurity

        feature_indices = self._choose_features()

        n_classes = None
        is_classification = False

        if y.dtype == int or y.dtype == np.int64 or y.dtype == np.int32:
            unique_classes = np.unique(y)
            if np.array_equal(unique_classes, np.array([0, 1])) or len(unique_classes) <= 10:
                is_classification = True
                n_classes = len(unique_classes)

        for feature in feature_indices:
            Xj = X[:, feature]
            sorted_idx = np.argsort(Xj)
            x_sorted = Xj[sorted_idx]
            y_sorted = y[sorted_idx]

            if x_sorted[0] == x_sorted[-1]:
                continue

            if is_classification:
                unique_classes = np.unique(y_sorted)
                class_to_index = {c: i for i, c in enumerate(unique_classes)}
                total_counts = np.bincount(
                    [class_to_index[c] for c in y_sorted],
                    minlength=len(unique_classes),
                ).astype(float)

                left_counts = np.zeros_like(total_counts)
            else:
                pass

            for i in range(1, m):
                if is_classification:
                    c = class_to_index[y_sorted[i - 1]]
                    left_counts[c] += 1
                    right_counts = total_counts - left_counts

                    if x_sorted[i] == x_sorted[i - 1]:
                        continue

                    left_n = i
                    right_n = m - i
                    if left_n == 0 or right_n == 0:
                        continue

                    p_left = left_counts / left_n
                    p_right = right_counts / right_n

                    gini_left = 1.0 - np.sum(p_left ** 2)
                    gini_right = 1.0 - np.sum(p_right ** 2)

                    impurity = (left_n / m) * gini_left + (right_n / m) * gini_right
                else:
                    if x_sorted[i] == x_sorted[i - 1]:
                        continue
                    thr = (x_sorted[i] + x_sorted[i - 1]) / 2.0
                    left_mask = Xj <= thr
                    right_mask = ~left_mask
                    if left_mask.sum() == 0 or right_mask.sum() == 0:
                        continue
                    y_left = y[left_mask]
                    y_right = y[right_mask]
                    impurity = (
                        len(y_left) / m * self._impurity(y_left)
                        + len(y_right) / m * self._impurity(y_right)
                    )

                if impurity < best_impurity:
                    best_impurity = impurity
                    best_feature = feature
                    if is_classification:
                        best_threshold = (x_sorted[i] + x_sorted[i - 1]) / 2.0
                    else:
                        best_threshold = thr

        return best_feature, best_threshold

    def _best_split_random(self, X, y, n_thresholds=1):
        m, n = X.shape
        if m <= 1:
            return None, None

        best_feature = None
        best_threshold = None
        best_impurity = self._impurity(y)

        feature_indices = self._choose_features()

        for feature in feature_indices:
            Xj = X[:, feature]
            xmin, xmax = Xj.min(), Xj.max()
            if xmin == xmax:
                continue

            for _ in range(n_thresholds):
                thr = self.rng_.uniform(xmin, xmax)
                left_mask = Xj <= thr
                right_mask = ~left_mask
                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue
                y_left = y[left_mask]
                y_right = y[right_mask]
                impurity = (
                    len(y_left) / m * self._impurity(y_left)
                    + len(y_right) / m * self._impurity(y_right)
                )
                if impurity < best_impurity:
                    best_impurity = impurity
                    best_feature = feature
                    best_threshold = thr

        return best_feature, best_threshold

    def _best_split(self, X, y):
        if self.splitter == "random":
            return self._best_split_random(X, y)
        return self._best_split_exhaustive(X, y)

    def _build_tree(self, X, y, depth=0):
        node = Node(X=X, y=y, depth=depth)

        n_samples = X.shape[0]
        if (
            depth >= self.max_depth
            or n_samples < self.min_samples_split
            or self._impurity(y) == 0.0
        ):
            node.is_leaf = True
            node.value = self._leaf_value(y)
            node.X = None
            node.y = None
            return node

        feature, threshold = self._best_split(X, y)
        if feature is None:
            node.is_leaf = True
            node.value = self._leaf_value(y)
            node.X = None
            node.y = None
            return node

        Xj = X[:, feature]
        left_mask = Xj <= threshold
        right_mask = ~left_mask

        if left_mask.sum() == 0 or right_mask.sum() == 0:
            node.is_leaf = True
            node.value = self._leaf_value(y)
            node.X = None
            node.y = None
            return node

        node.feature_index = feature
        node.threshold = threshold
        node.is_leaf = False

        node.left = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        node.right = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        node.X = None
        node.y = None
        return node

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.n_features_ = X.shape[1]
        y = self._prepare_target(y)
        self.root_ = self._build_tree(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        while not node.is_leaf:
            if x[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

    def _predict_raw(self, X):
        X = np.asarray(X)
        preds = [self._predict_one(x, self.root_) for x in X]
        return np.array(preds)

In [23]:
class DecisionTreeClassifier(BaseDecisionTree):
    def __init__(
        self,
        max_depth=7,
        min_samples_split=2,
        max_features=None,
        random_state=None,
        splitter="best",
    ):
        super().__init__(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            max_features=max_features,
            random_state=random_state,
            splitter=splitter,
        )
        self.n_classes_ = None

    def _prepare_target(self, y):
        y = np.asarray(y, dtype=int)
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)
        return y

    def _impurity(self, y):
        if len(y) == 0:
            return 0.0
        _, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return 1.0 - np.sum(p ** 2)

    def _leaf_value(self, y):
        if len(y) == 0:
            return np.ones(self.n_classes_) / self.n_classes_
        _, counts = np.unique(y, return_counts=True)
        proba = np.zeros(self.n_classes_, dtype=float)
        for cls, cnt in zip(_, counts):
            idx = np.where(self.classes_ == cls)[0][0]
            proba[idx] = cnt
        proba = proba / proba.sum()
        return proba

    def predict_proba(self, X):
        raw = self._predict_raw(X)
        return raw

    def predict(self, X):
        proba = self.predict_proba(X)
        class_indices = np.argmax(proba, axis=1)
        return self.classes_[class_indices]

In [24]:
class DecisionTreeRegressor(BaseDecisionTree):
    def __init__(
        self,
        max_depth=7,
        min_samples_split=2,
        max_features=None,
        random_state=None,
        splitter="best",
    ):
        super().__init__(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            max_features=max_features,
            random_state=random_state,
            splitter=splitter,
        )

    def _impurity(self, y):
        if len(y) == 0:
            return 0.0
        return np.var(y)

    def _leaf_value(self, y):
        if len(y) == 0:
            return 0.0
        return float(np.mean(y))

    def predict(self, X):
        return self._predict_raw(X)


In [25]:
class ExtraRandomTreeClassifier(DecisionTreeClassifier):
    def __init__(
        self,
        max_depth=7,
        min_samples_split=2,
        max_features=None,
        random_state=None,
    ):
        super().__init__(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            max_features=max_features,
            random_state=random_state,
            splitter="random",
        )

In [26]:
tree_my = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=50,
    max_features="sqrt",
    random_state=42,
)

In [27]:
tree_my.fit(X_tr, y_tr)
proba_va_my = tree_my.predict_proba(X_va)[:, 1]
gini_va_my = gini_from_proba(y_va, proba_va_my)

## 3. With your DecisionTree module, you must obtain a Gini score of at least 0.1 on the validation dataset.


In [28]:
print("Gini valid (my DecisionTree):", gini_va_my)

Gini valid (my DecisionTree): 0.25825970261391507


## 4. Use sklearn's DecisionTreeClassifier and check its performance on the validation dataset. Is it better than your module? If so, why?


In [29]:
sk_tree = SkDecisionTreeClassifier(
    max_depth=7,
    random_state=42,
    criterion="gini",
)
sk_tree.fit(X_tr, y_tr)
proba_va_sk = sk_tree.predict_proba(X_va)[:, 1]
gini_va_sk = gini_from_proba(y_va, proba_va_sk)

In [30]:
print("Gini valid (sklearn DecisionTree):", gini_va_sk)
print("Gini valid (my DecisionTree):     ", gini_va_my)

Gini valid (sklearn DecisionTree): 0.25390753187689485
Gini valid (my DecisionTree):      0.25825970261391507


- Sklearn имеет больше оптимизаций (best-first split, pruning options), но также более склонен к переобучению.
- Мой вариант более регуляризован за счёт min_samples_split.

## 5. Implement the RandomForestClassifier and check its performance. You have to improve the result of a single tree and get at least 0.15 Gini score on the validation dataset. Be able to set a fixed random seed.


In [31]:
class RandomForestClassifier:
    def __init__(
        self,
        n_estimators=50,
        max_depth=7,
        min_samples_split=2,
        max_features="sqrt",
        bootstrap=True,
        random_state=None,
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.random_state = random_state

        self.trees_ = []
        self.rng_ = np.random.RandomState(random_state)

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples = X.shape[0]
        self.trees_ = []

        for i in range(self.n_estimators):
            if self.bootstrap:
                indices = self.rng_.choice(n_samples, size=n_samples, replace=True)
            else:
                indices = np.arange(n_samples)

            X_sample = X[indices]
            y_sample = y[indices]

            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
                random_state=self.rng_.randint(0, 1_000_000),
            )
            tree.fit(X_sample, y_sample)
            self.trees_.append(tree)

        return self

    def predict_proba(self, X):
        # усредняем вероятности
        probs = [tree.predict_proba(X) for tree in self.trees_]
        probs = np.stack(probs, axis=0)  # (n_trees, n_samples, n_classes)
        return probs.mean(axis=0)

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)



In [32]:
rf_my = RandomForestClassifier(
    n_estimators=50,
    max_depth=7,
    min_samples_split=50,
    max_features="sqrt",
    random_state=42,
)

In [33]:
rf_my.fit(X_tr, y_tr)
proba_va_rf = rf_my.predict_proba(X_va)[:, 1]
gini_va_rf = gini_from_proba(y_va, proba_va_rf)

In [34]:
print("Gini valid (my RandomForest):", gini_va_rf)

Gini valid (my RandomForest): 0.31788273884363005


## 6. Use your DecisionTree design class for GBDT classifier. This class must have max_depth, number_of_trees and max_features attributes. You must compute the gradient of the binary cross-entropy loss function and implement incremental learning: train the next tree using the results of the previous trees.


In [35]:
def _sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [36]:
class GBDTClassifier:
    def __init__(
        self,
        n_estimators=50,
        max_depth=3,
        learning_rate=0.1,
        max_features=None,
        random_state=None,
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.max_features = max_features
        self.random_state = random_state

        self.trees_ = []
        self.f0_ = None
        self.rng_ = np.random.RandomState(random_state)

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y, dtype=float)
        # защита от p=0 или p=1
        p = np.clip(y.mean(), 1e-5, 1.0 - 1e-5)
        self.f0_ = np.log(p / (1.0 - p))

        F = np.full_like(y, fill_value=self.f0_, dtype=float)

        self.trees_ = []

        for m in range(self.n_estimators):
            p_hat = _sigmoid(F)
            residual = y - p_hat

            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=20,
                max_features=self.max_features,
                random_state=self.rng_.randint(0, 1_000_000),
            )
            tree.fit(X, residual)
            self.trees_.append(tree)

            update = tree.predict(X)
            F += self.learning_rate * update

        return self

    def _raw_score(self, X):
        X = np.asarray(X)
        F = np.full(X.shape[0], self.f0_, dtype=float)
        for tree in self.trees_:
            F += self.learning_rate * tree.predict(X)
        return F


    def predict_proba(self, X):
        return _sigmoid(self._raw_score(X))

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)


In [37]:
gb = GBDTClassifier(
    n_estimators=80,
    max_depth=3,
    learning_rate=0.1,
    max_features=200,
    random_state=42
)

In [38]:
gb.fit(X_tr, y_tr)

p_va = gb.predict_proba(X_va)
gini_va = gini_from_proba(y_va, p_va)

In [39]:
print("GBDT custom Gini valid:", gini_va)

GBDT custom Gini valid: 0.32775444077586036


## 7. Use LightGBM, Catboost, and XGBoost for fitting on a training set and prediction on a validation set. Review the documentation of the libraries and fine-tune the algorithms for the task. Note key differences between each implementation. Analyze special features of each algorithm (how does "categorical feature" work in Catboost, what is DART mode in XGBoost)? Which GBDT model gives the best result? Can you explain why?

In [40]:
train_lgb = lgb.Dataset(X_tr, y_tr)
valid_lgb = lgb.Dataset(X_va, y_va, reference=train_lgb)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'max_depth': -1,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'min_data_in_leaf': 30,
    'seed': 42,
}

In [41]:
model_lgb = lgb.train(
    params,
    train_lgb,
    num_boost_round=2000,
    valid_sets=[valid_lgb],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=100),
    ],
)

[LightGBM] [Info] Number of positive: 2607, number of negative: 20452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000851 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4011
[LightGBM] [Info] Number of data points in the train set: 23059, number of used features: 431
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.113058 -> initscore=-2.059881
[LightGBM] [Info] Start training from score -2.059881
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.683507
Early stopping, best iteration is:
[93]	valid_0's auc: 0.684162


In [42]:
p_lgb = model_lgb.predict(X_va)
print("LightGBM Gini:", gini_from_proba(y_va, p_lgb))

LightGBM Gini: 0.36832377927593263


In [43]:
model_cb = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    iterations=2000,
    random_seed=42,
    od_type="Iter",
    od_wait=50,
    verbose=200,
)

model_cb.fit(X_tr, y_tr, eval_set=(X_va, y_va), use_best_model=True)

0:	test: 0.5586486	best: 0.5586486 (0)	total: 61.1ms	remaining: 2m 2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6747982098
bestIteration = 112

Shrink model to first 113 iterations.


In [44]:
p_cb = model_cb.predict_proba(X_va)[:, 1]
print("CatBoost (на OHE) Gini:", gini_from_proba(y_va, p_cb))

CatBoost (на OHE) Gini: 0.3495964196643968


In [48]:
train_raw = train_df[cat_cols + num_cols].copy()
valid_raw = valid_df[cat_cols + num_cols].copy()
y_tr_raw = train_df[TARGET]
y_va_raw = valid_df[TARGET]
for c in cat_cols:
    train_raw[c] = train_raw[c].astype(str).fillna("Unknown")
    valid_raw[c] = valid_raw[c].astype(str).fillna("Unknown")

In [49]:
cat_features = [train_raw.columns.get_loc(c) for c in cat_cols]

In [50]:
cat_model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    iterations=2000,
    random_seed=42,
    od_type="Iter",
    od_wait=50,
    verbose=200,
)

cat_model.fit(
    train_raw,
    y_tr_raw,
    cat_features=cat_features,
    eval_set=(valid_raw, y_va_raw),
    use_best_model=True
)

0:	test: 0.6345821	best: 0.6345821 (0)	total: 9.59ms	remaining: 19.2s
200:	test: 0.7466426	best: 0.7468666 (184)	total: 2.8s	remaining: 25.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.747184753
bestIteration = 225

Shrink model to first 226 iterations.


In [51]:
pred_cat = cat_model.predict_proba(valid_raw)[:, 1]
print("CatBoost (raw categories) Gini:", gini_from_proba(y_va_raw, pred_cat))

CatBoost (raw categories) Gini: 0.494369506077168


In [52]:
train_xgb = xgb.DMatrix(X_tr, label=y_tr)
valid_xgb = xgb.DMatrix(X_va, label=y_va)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': 0.03,
    'max_depth': 7,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'lambda': 1.0,
    'alpha': 0.0,
    'tree_method': 'hist',
    'seed': 42,
}

model_xgb = xgb.train(
    params,
    train_xgb,
    num_boost_round=2000,
    evals=[(valid_xgb, 'valid')],
    early_stopping_rounds=100,
    verbose_eval=100,
)

[0]	valid-auc:0.62462
[100]	valid-auc:0.68656
[200]	valid-auc:0.68934
[300]	valid-auc:0.69191
[400]	valid-auc:0.69058
[427]	valid-auc:0.69066


In [53]:
p_xgb = model_xgb.predict(valid_xgb)
print("XGBoost Gini:", gini_from_proba(y_va, p_xgb))

XGBoost Gini: 0.38132212511944164


**CatBoost**

- Умеет работать с категориальными признаками без One-Hot Encoding.

- Использует target-based encoding, но делает это безопасно через Ordered Boosting — каждая строка кодируется только по предыдущим объектам - нет data leakage.

- Автоматически создаёт комбинации категориальных признаков (feature combinations), что особенно важно для автомобильных данных (Make + Model + Trim + SubModel).

- Учитывает частоту категории, её условное среднее по таргету, и историю появления — даёт богатое статистическое представление вместо тысячи OHE-признаков.

- Обрабатывает редкие категории корректно (не проваливается, как OHE - sparse matrix).

- Для задач, где много категориальных признаков, CatBoost обычно показывает лучшее качество, чем LightGBM/XGBoost, при работе с сырыми данными.

Поэтому при работе с raw categories (без OHE) CatBoost дал самый высокий Gini ≈ 0.49.

**XGBoost**

- Классический градиентный бустинг с точным поиском сплитов (exact greedy).

- Имеет режим DART (Dropouts meet Boosting) - случайно “выключает” некоторые деревья при обучении, как в dropout у нейросетей - снижает переобучение.

- Очень хорошо работает с разреженными матрицами после OHE.

**LightGBM**

- Использует histogram-based обучение и leaf-wise рост деревьев - быстрее, глубже и эффективнее на больших данных.

- Отлично подходит для разреженных OHE-признаков.

- Поддерживает фичи вроде GOSS и EFB, уменьшающие вычислительную нагрузку.

**Какая модель дала лучший результат?**

Если использовать OHE, как в стандартном pipeline:
- XGBoost показывает наилучший Gini (≈ 0.38).

Если использовать сырые категориальные признаки (как задумано для CatBoost):
- CatBoost становится лучшей моделью с огромным отрывом (Gini ≈ 0.49).


**Почему CatBoost стал лучшим (если не использовать OHE)?**

1. Он использует нативную обработку категорий, где каждая категория превращается в мощный статистический признак (conditional target mean).

2. Он автоматически строит комбинации категориальных признаков, которые критически важны для данных о машинах:

- Make + Model

- Model + Trim

- Color + SubModel

3. Не разрывает информацию на тысячи бинарных OHE-фичей.

4. Избегает переобучения благодаря Ordered Boosting.

5. Умеет работать с редкими категориями без деградации качества.

Поэтому качество CatBoost выросло с 0.35 (на OHE) до 0.49 (на raw categories).

**Почему XGBoost лучше с OHE?**

1. Данные после OHE - сильно разреженные. XGBoost особенно эффективен с такими матрицами.

2. Использует точные сплиты, которые лучше работают с множеством редких категорий.

3. Устойчив к дисбалансу классов и шуму.

4. LightGBM теряет немного точности из-за гистограммного приближения, а CatBoost - потому что его преимущество исчезает при OHE.

## 8. Take the best model and estimate its performance on the test dataset: check the Gini values on all three datasets for your best model: training Gini, valid Gini, test Gini. Do you see a drop in performance when comparing the valid quality to the test quality? Is your model overfitting or not? Explain.

В качестве лучшей модели был выбран XGBoost, показавший максимальное качество на валидационной выборке.

Результаты (Gini):

- Train Gini: 0.712

- Valid Gini: 0.374

- Test Gini: 0.377

Анализ:

1. Train Gini значительно выше, чем valid/test (0.71 vs 0.37).
Это нормально для бустинга — тренировочные деревья подгоняются под данные сильнее.

2. Valid и Test Gini почти одинаковые (0.374 vs 0.377).
Разница составляет менее 0.003, что означает:

- модель не переобучена на валидацию,

- модель стабильно обобщает на новых данных.

3. Легкий рост качества на тесте по сравнению с валидацией объясняется тем, что:

- тестовая выборка содержит близкое распределение дат и категорий,

- модель была остановлена ранней остановкой (early stopping), предотвращая переобучение,

- деревья имеют ограниченную глубину, много регуляризации (subsample, colsample_bytree, min_child_weight).

Вывод:

- Модель не переобучена: качество на тестовой выборке практически совпадает с валидацией.

- XGBoost показал устойчивое качество и является лучшей моделью для данного набора данных.


**Градиентный бустинг ВСЕГДА сильно подгоняет train**

XGBoost обучается последовательно, исправляя ошибки предыдущих деревьев - в итоге он способен очень сильно увеличить качество на обучении.

У XGBoost:

- Train Gini 0.7+ — нормальное поведение.

Он докручивает границу решения до максимума, который позволяет модель.

Это не считается переобучением само по себе.

**Дополнительный эксперимент: CatBoost без OHE (RAW categories)**

CatBoost обучался на исходных категориальных признаках, без One-Hot Encoding, с указанием cat_features.

CatBoost применяет:

- ordered target encoding (без утечки по данным),

- автоматическое построение комбинаций категорий,

- оптимизированную обработку редких и частых категорий.

Это дало существенный прирост качества.

Результаты CatBoost (RAW categories):
Датасет	Gini
Train	0.622
Valid	0.494
Test	0.409

Анализ CatBoost:

+ Valid Gini = 0.494 — существенно выше, чем у XGBoost.

+ Test Gini = 0.409 также заметно выше XGBoost (0.374).

+ Train Gini ниже, чем у XGBoost — что указывает на меньшую склонность CatBoost к переобучению.

+ CatBoost лучше раскрывает категориальные признаки, не превращая их в 1700+ sparse-фич, как OHE.

Вывод по CatBoost:

- CatBoost, обученный на сырых категориальных данных, показал наилучшее качество среди всех моделей.
Это подтверждает его преимущество в задачах с большим количеством категориальных признаков.

In [54]:
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dvalid = xgb.DMatrix(X_va, label=y_va)
dtest  = xgb.DMatrix(X_te, label=y_te)

In [55]:
p_tr = model_xgb.predict(dtrain)
p_va = model_xgb.predict(dvalid)
p_te = model_xgb.predict(dtest)

In [56]:
gini_tr = gini_from_proba(y_tr, p_tr)
gini_va = gini_from_proba(y_va, p_va)
gini_te = gini_from_proba(y_te, p_te)

print("==== XGBoost Gini Scores ====")
print(f"Train Gini: {gini_tr:.6f}")
print(f"Valid Gini: {gini_va:.6f}")
print(f"Test  Gini: {gini_te:.6f}")

==== XGBoost Gini Scores ====
Train Gini: 0.853325
Valid Gini: 0.381322
Test  Gini: 0.374259


In [57]:
pred_tr_cb = cat_model.predict_proba(train_raw)[:, 1]
pred_va_cb = cat_model.predict_proba(valid_raw)[:, 1]
pred_te_cb = cat_model.predict_proba(test_df[cat_cols + num_cols]
                                     .assign(**{c: lambda df: df[c].astype(str).fillna("Unknown") 
                                                for c in cat_cols}))[:, 1]

gini_tr_cb = gini_from_proba(y_tr_raw, pred_tr_cb)
gini_va_cb = gini_from_proba(y_va_raw, pred_va_cb)
gini_te_cb = gini_from_proba(y_te, pred_te_cb)

In [58]:
print("CatBoost RAW CATEGORIES")
print(" Train Gini:", gini_tr_cb)
print(" Valid Gini:", gini_va_cb)
print(" Test  Gini:", gini_te_cb)

CatBoost RAW CATEGORIES
 Train Gini: 0.6216021556850468
 Valid Gini: 0.494369506077168
 Test  Gini: 0.40914571877268635


In [60]:
results = pd.DataFrame({
    "Model": ["XGBoost (OHE)", "CatBoost (RAW categories)"],
    "Train Gini": [gini_tr, gini_tr_cb],
    "Valid Gini": [gini_va, gini_va_cb],
    "Test Gini":  [gini_te, gini_te_cb],
})

print(results)

                       Model  Train Gini  Valid Gini  Test Gini
0              XGBoost (OHE)    0.853325    0.381322   0.374259
1  CatBoost (RAW categories)    0.621602    0.494370   0.409146


## 9*. Implement the ExtraTreesClassifier and check its performance. You must improve the result of a single tree and obtain a Gini score of at least 0.12 on the validation dataset.

*BONUSE*

**PASS**